## A few tips for more efficient coding

In [1]:
# Unless otherwise stated, from now on we're going to eliminate spaces from our plaintext and only
# work with continuous strings of characters in the {a, b, ..., z} range.

# A simple way to eliminate spaces from a string is to use the replace() method
s = 'My PlainText String'
s = s.replace(' ','')
print(s)

# note that "replace" can be used for replacing any character with another character in a string
# Other useful methods for use on string variables are upper() and lower() methods which convert all characters in a string 
# to uppercase or lowercase, respectively
print(s.lower())

MyPlainTextString
myplaintextstring


In [2]:
# We also showed in class how Python's list comprehensions can be used to efficiently convert strings to indices
s = 'myplaintextstring'
s_ind_list = [ord(c)-97 for c in s]
print(s_ind_list)

[12, 24, 15, 11, 0, 8, 13, 19, 4, 23, 19, 18, 19, 17, 8, 13, 6]


In [3]:
# The reverse operation is also possible
char_list = [chr(ind+97) for ind in s_ind_list]
print(char_list)
# The join method creates a string by "joining" all elements of the given list together using the provided character
# In this case the provided character is null so all elements are just directly attached together to give us s
s = ''.join(char_list)
print(s)

['m', 'y', 'p', 'l', 'a', 'i', 'n', 't', 'e', 'x', 't', 's', 't', 'r', 'i', 'n', 'g']
myplaintextstring


In [4]:
# Another tip is for when we need to work on chunks of a list (for example in Hill 3x3 cipher we need
# to select the elements of the plaintext in chunks of 3)
# we can use the range function with the step of 3
for i in range(0,len(char_list),3):  # it means integers from 0 to len(char_list)-1 with step 3
    sublist = char_list[i:i+3]  # it means elements i,i+1,i+2 (the last element -1)
    print(sublist)

['m', 'y', 'p']
['l', 'a', 'i']
['n', 't', 'e']
['x', 't', 's']
['t', 'r', 'i']
['n', 'g']


## Finding mod-26 inverse of a matrix

### We mentioned that the modulo 26 inverse of a matrix may noit exists. But if it does, we can make use
### of the normal inverse and determinent of a matrix calculations given by some math packages to find the
### mod-26 inverse of a matrix
### Our Notation (We focus on 3x3 matrices, but this logic works for larger matrices as well):
### R: set of all real numbers
### M: original matrix with elements in range 0-25 (a 3x3 matrix)
### Minv: real inverse of M (a 3x3 matrix, values in R)
### Mdet: real determinant of M (a number, in R)
### Madj: real adjoint matrix of M (3x3 matrix, values in R)

### Similarly, we use a 26 suffix to show the equivalent of the above values in mod-26 space i.e.,
### Minv26: mod-26 inverse of M (a 3x3 matrix, values in 0-25)
### Mdet26: mod-26 determinant of M (a number in 0-25)
### Mdetinv26: mod-26 inverse Mdet26 (a number in 0-25)
### Madj26: mod-26 adjoint matrix of M (3x3 matrix, values in 0-25)

### For real matrices we have the following equation for finding the inverse matrix:
### Minv = (1/Mdet) * Madj     (Eq. 2)

### But for mod-26 matrices:
### Minv26 = (Mdetinv26 *  Madj26)%26     (Eq. 1)


In [5]:
# Let's first see how Mdetinv26 is defined for equation (1) above. The mod-26 inverse of a number m between 0-25 is another number n 
# in that range such that
# m*n mod 26 = 1
# For example the inverse of 3 is 9 since 3*9 mod 26 = 27 mod 26 = 1.
# Not all 0-25 numbers have a mod-26 inverse. Let's try to find those that have inverses
Mod26invTable = {}
for m in range(26):
    for n in range(26):
        if (m*n)%26==1:
            Mod26invTable[m] = n
            print(m,n)

# So, if we are lucky and Mdet26 is one of the above integers, it has an inverse and we can find it from the 
# above table we just constructed.
# For example, inverse of 3 is stored in Mod26invTable[3]

# In other words, if a python function, gives us the real determinant Mdet of M, we can find Mdetinv26 from:
# Mdetinv26 = Mod26invTable[Mdet%26]  (provided that Mdet%26 has an inverse)


1 1
3 9
5 21
7 15
9 3
11 19
15 7
17 23
19 11
21 5
23 17
25 25


In [6]:
# Let's now see how we can compute Madj26 for equatin (1).
# Madj26 is actually Madj mod 26 so we need to find Madj
# The numpy module in Python has functions for Minv and Mdet and we can always find Madj form Minv and Mdet 
# using equation (2) above:
# Madj = Mdet * Minv, therefore
# Madj26 = (Mdet*Minv)%26
import numpy as np
from pprint import pprint

M = np.array([[17,17,5],[21,18,21],[2,2,19]])
print("M: ")
print(M)
Minv = np.linalg.inv(M)
Mdet = np.linalg.det(M)

# Let's find Mdet26 and Mdetinv26
Mdet26 = Mdet%26
if Mdet26 in Mod26invTable:
    Mdetinv26 = Mod26invTable[Mdet26]
else:
    Mdetinv26 = None # This should be an exit point since we can't find an inverse for M in mod-26

#print(Mdet,Mdetinv26)

# Now, Let's find Madj26
Madj = Mdet*Minv
Madj26 = Madj%26


#print(np.matmul(M,Minv))

#So, The mod-26 inverse of M from equation (1) will be
Minv26 = (Mdetinv26*Madj26)%26
# We need to convert it to pure integers. So we round the elments and then do mod-26 again
Minv26 = np.matrix.round(Minv26, 0)%26

print("Minv26:")
print(Minv26)

# Let's check and see if we did right:
# M x Minv26 should be I (we need some rounding and %26 operations)
MMinv26 = np.matmul(M,Minv26)%26
MMinv26 = np.matrix.round(MMinv26,0)

print("M x Minv26: ")
print(MMinv26)
# If we wish to convert a matrix back from numpy format to basic list-of-lists format, 
# we can use the tolist() method:
print(MMinv26.tolist())

# # Mcr = np.matrix.round(Minv % 26, 0)%26
# print(Mdet)

M: 
[[17 17  5]
 [21 18 21]
 [ 2  2 19]]
Minv26:
[[ 4.  9. 15.]
 [15. 17.  6.]
 [24.  0. 17.]]
M x Minv26: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]


# Homework 2- Part 1: Vigenere Cipher
## Write an encryption and a decryption function for Vigenere cipher as described below

In [7]:
# Hint: You can use your homework1 caesar encryption and decryption functions but you need to copy them
#       here and avoid importing that homework since we won't be able to do the same thing for grading.
# We will be importing this python file and call its functions in another grading script. So we don't need any
# command line argument support like what we did for the caesar cipher.

def ch_enc(ch, V):
    val = ord(ch)
    K = ord(V) - 97
    enc_ch = chr((val + K - 97)%26 + 97)
    return enc_ch

def ch_dec (ch, V):
    val = ord(ch)
    K = ord(V) - 97
    dec_ch = chr((val - K - 97)%26 + 97)
    return dec_ch

# Vigenere encryption function
def vigenere_enc(keyword, plaintext):
    # keyword is a string of arbitrary length
    # plaintext is the plaintext string of arbitrary length
    # Both strings will be from {a,b,...,z}
    
    # perform the encryption of given plaintext using the given keyword
    # according to the Vigenere cipher. You need to repeat the keyword 
    # enough times if needed to make it the same length as plaintext
    
    # c will be the resulting ciphertext
    #c = ...
    c = ''
    for i in range(0, len(plaintext)):
        c = c + ch_enc(plaintext[i], keyword[i%len(keyword)])
    
    return c


# Vionegere decryption function
def vigenere_dec(keyword, ciphertext):
    # keyword is a string of arbitrary length
    # ciphertext is the ciphertext string of arbitrary length
    # Both strings will be from {a,b,...,z}
    
    # perform the decryption of given ciphertext using the given keyword
    # according to the Vigenere cipher. You need to repeat the keyword 
    # enough times if needed to make it the same length as ciphertext
    
    # p will be the resulting plaintext
    # p = ...
    p = ''
    for i in range(0, len(ciphertext)):
        p = p + ch_dec(ciphertext[i], keyword[i%len(keyword)])
    
    return p
    

# Homework 2- Part 2: Hill Cipher
## Write an encryption and a decryption function for Hill cipher as described below

In [8]:
# Use the discussion above for finding the mod-26 inverse of a matrix and write the encryption and decryption 
# functions for a 3x3 Hill cipher.
import numpy as np

# 1- clean up the inverting operations explained above to get a matrix inversion-mod-26 function
def matrixinvmod26(Marr):
    # Both the input argument M an doutput Minv26 are in the list of lists format i.e.,
    # [[M11,M12,M13],[M21,M22,M23],[M31,M32,M33]]
    # use np.array() and tolist() functions as described above for conversion between matrix and list-of-lists 
    
    # Calculate Minv26
    Minv = np.linalg.inv(Marr)
    Mdet = np.linalg.det(Marr)
    
    Mod26invTable = {}
    for m in range(26):
        for n in range(26):
            if (m*n)%26==1:
                Mod26invTable[m] = n
    
    Mdet26 = Mdet%26
    if Mdet26 in Mod26invTable:
        Mdetinv26 = Mod26invTable[Mdet26]
    else:
        Mdetinv26 = None
        print("No inverse found")
        exit()
    
    Madj = Mdet*Minv
    Madj26 = Madj%26
    Minv26 = (Mdetinv26*Madj26)%26
    Minv26 = np.matrix.round(Minv26, 0)%26
    
    return Minv26

# 2- write the Hill encryption function 
def hill_enc(M, plaintext):
    # M is the encryption matrix - Let's assume it's always 3x3 for now
    # M is in the list of lists format i.e.,
    # [[M11,M12,M13],[M21,M22,M23],[M31,M32,M33]]
    # plaintext is the plaintext string of arbitrary length
    # from {a,b,...,z}
    
    # perform the encryption of given plaintext using the given matrix M
    # according to the Hill cipher. Pad the plaintext with 'x' characters to 
    # make its length a multiple of 3.     
    
    # Some helpful funcitons:
    # len(plaintext) : gives the length of the plaintext string
    # one way of selecting chunks of 3 elements from a list:
    # 
    
    # c will be the resulting ciphertext
    #c = ...
    c = ''
    tmp = []
    if len(plaintext)%3 != 0:
        plaintext = plaintext + 'x'*(3-len(plaintext)%3)
        
#     print("Plain text after padding", plaintext)
        
    plaintext_list = [ord(p)-97 for p in plaintext]
#     print("plaintext_list= ", plaintext_list)
    Marr = np.array(M)
#     print("Key array", Marr)
    
    for i in range(0, len(plaintext_list), 3):
        sublist = plaintext_list[i:i+3]
#         print("sublist= ", sublist)
        subarr = np.array(sublist)
#         print("subarr= ", subarr)
        subarr_enc = np.matrix.round(np.matmul(M, subarr)%26, 0)%26
#         print("subarr_enc= ", subarr_enc)
        sublist_enc = subarr_enc.tolist()
#         print("sublist_enc= ", sublist_enc)
        tmp.extend(sublist_enc)
#         print("tmp= ", tmp)
    
    c = c.join(chr(x+97) for x in tmp)
    return c


# 3- write the Hill decryption function
def hill_dec(M, ciphertext):
    # M is the encryption matrix - Let's assume it's always 3x3 for now
    # M is in the list of lists format i.e.,
    # [[M11,M12,M13],[M21,M22,M23],[M31,M32,M33]]
    # ciphertext is the ciphertext string of arbitrary length
    # from {a,b,...,z}
    
    # perform the decryption of given ciphertext using the given matrix M
    # according to the Hill cipher. 
    
    # p will be the resulting plaintext
    # p = ...
    p = ''
    tmp = []
    
    ciphertext_list = [ord(x)-97 for x in ciphertext]
#     print("ciphertext_list= ", ciphertext_list)
    
    Marr = np.array(M)
#     print("Key array= ", Marr)
    Minv = matrixinvmod26(Marr)
#     print("Key inverse array= ", Minv)
    
    for i in range(0, len(ciphertext_list), 3):
        sublist = ciphertext_list[i:i+3]
#         print("sublist= ", sublist)
        subarr = np.array(sublist)
#         print("subarr= ", subarr)
        subarr_dec = np.matrix.round(np.matmul(Minv, subarr)%26, 0)%26
#         print("subarr_dec= ", subarr_dec)
        sublist_dec = subarr_dec.tolist()
#         print("sublist_dec= ", sublist_dec)
        tmp.extend(sublist_dec)
#         print("tmp= ", tmp)
        
    p = p.join(chr(int(x)+97) for x in tmp)
    return p
    
enct = hill_enc([[17,17,5],[21,18,21],[2,2,19]], 'anush')
print(enct)
print(hill_dec([[17,17,5],[21,18,21],[2,2,19]], enct))

jequzt
anushx
